In [1]:
# Baslerカメラで撮影、さらにフィードバックループで刺激を行うためののGUI
# USBカメラにも対応させる！
# Bpodからのキューで撮影開始、終了を操作する

import os
import re
import cv2
import sys
import glob
import shutil
import time
import datetime
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from skimage.feature import hog
from threading import Thread
import serial
from serial.tools import list_ports
import socket
from tqdm import tqdm

import subprocess
import multiprocessing

from pypylon import pylon
from PIL import Image, ImageTk
from dlclive import DLCLive, Processor

from utils import *
from utilsCamera import *
from utilsGUI import *
import utilsGUI
from config import dict_config

# 並列実行用
import functools
from concurrent.futures import ProcessPoolExecutor, as_completed
from forMultiProcessing import hogFeatures

# PC名
pc_host = socket.gethostname()
dict_config_pc = dict_config[pc_host]

class CameraGUI:
    def __init__(self, debug=True, toplevel=False):
        # USBカメラかBaslerカメラか選択
#         self.chooseCameraName(list_camera=["USB", "Basler"], toplevel=toplevel)
        self.camera_name = "Basler"
        initializeWindow(self, width=1920, height=900, title="BaslerCameraGUI", debug=debug, toplevel=toplevel)

        pad = self.pad
        entry_width = self.entry_config_width
        
        # フレームの作成
        self.frame_dict["config"] = tk.Frame(self.frame_dict["main"], bd=2, relief="groove")
        self.frame_dict["capture"] = tk.Frame(self.frame_dict["main"], bd=2, relief="groove")
        self.frame_dict["image"] = tk.Frame(self.frame_dict["main"], bd=2, relief="groove")
        
        # フレームの配置
        self.frame_dict["config"].grid(row=0, column=0, padx=pad, pady=pad, sticky="nw")
        self.frame_dict["capture"].grid(row=1, column=0, padx=pad, pady=pad, sticky="nw")
        self.frame_dict["image"].grid(row=0, column=1, sticky="nw", rowspan=2, padx=pad, pady=pad)
        

        # --- frame_dict["config"] ---
        # 内部フレームの作成、配置
        self.frame_dict["config_save"] = tk.Frame(self.frame_dict["config"])
        self.frame_dict["config_serial"] = tk.Frame(self.frame_dict["config"])
        self.frame_dict["config_func"] = tk.Frame(self.frame_dict["config"])
        self.frame_dict["config_camera"] = tk.Frame(self.frame_dict["config"])
        self.frame_dict["config_save"].grid(row=0, column=0)
        self.frame_dict["config_serial"].grid(row=1, column=0)
        self.frame_dict["config_func"].grid(row=2, column=0)
        self.frame_dict["config_camera"].grid(row=3, column=0)
        
        # "Save Directory"項目の作成
        createEntry(key="savedir", frame=self.frame_dict["config_save"], entry_dict=self.entry_dict, 
                       row=0, column=0, text="Save Directory:", entry_width=50, pad=pad,
                       textvariable = tk.StringVar(value=dict_config_pc["savedir_camera"]))
        createButton(key="savedir", frame=self.frame_dict["config_save"], button_dict=self.button_dict,
                          row=0, column=2, text="browse", 
                          command=lambda: chooseDir(self.entry_dict["savedir"]))
        # 選択するprefix
        pulldown_prefix_value = ["dlc-pupil", "HoG-ActiveWhisking", "HoG-Sniffing"]
        createPulldown(key="savedir_prefix", frame=self.frame_dict["config_save"], pulldown_dict=self.pulldown_dict, 
                       row=1, column=0, text="Save Directory Prefix (ex) dlc-pupil :", pulldown_width=50, values=pulldown_prefix_value)

        # "Move Directory"項目の作成
        createEntry(key="movedst", frame=self.frame_dict["config_save"], entry_dict=self.entry_dict, 
                               row=2, column=0, text="Move Destination:", entry_width=50, pad=pad,
                               textvariable = tk.StringVar(value=dict_config_pc["movedst_camera"]))
        createButton(key="movedst", frame=self.frame_dict["config_save"], button_dict=self.button_dict,
                          row=2, column=2, text="browse", 
                          command=lambda: chooseDir(self.entry_dict["movedst"]))
        
        # シリアル通信の設定
        createLabel(key="config_serial", frame=self.frame_dict["config_serial"], label_dict=self.label_dict, 
                    row=0, column=0, text="Serial Config:")
        createEntry(key="baudrate", frame=self.frame_dict["config_serial"], entry_dict=self.entry_dict, 
                    row=1, column=0, text="baudrate:", entry_width=20, pad=pad, textvariable=tk.IntVar(value=9600))
        createPulldown(key="comport_receiver", frame=self.frame_dict["config_serial"], pulldown_dict=self.pulldown_dict, 
                       row=1, column=2, text="COM port Receiver:", values=[info.device for info in list_ports.comports()],
                       textvariable=dict_config_pc["comport"]["camera_receiver"])
        createPulldown(key="comport_sender", frame=self.frame_dict["config_serial"], pulldown_dict=self.pulldown_dict, 
                       row=2, column=2, text="COM port Sender:", values=[info.device for info in list_ports.comports()],
                       textvariable=dict_config_pc["comport"]["camera_sender"])
        
        # Closed loopで実行する関数
        createLabel(key="config_function", frame=self.frame_dict["config_func"], label_dict=self.label_dict, 
                         row=0, column=0, text="Function Config:")

        self.dict_func = {"wait100msec": self.wait100msec,
                          "valuePixel": self.valuePixel,
                          "valueHoGVectorL2normDelta": self.valueHoGVectorL2normDelta,
                          "dlcLivePupil": self.dlcLivePupil,}
        createPulldown(key="function_loop", frame=self.frame_dict["config_func"], pulldown_dict=self.pulldown_dict, 
                            row=1, column=0, text="Function for Closed-Loop:", values=list(self.dict_func.keys()),
                            pulldown_width=50, columnspan=3)
        # crop coordsの指定ボタン
        createEntry(key="cropcoords", frame=self.frame_dict["config_func"], entry_dict=self.entry_dict, 
                       row=2, column=0, text="cropcoords (xmin, xmax, ymin, ymax):", entry_width=entry_width, pad=pad)
        createButton(key="set_cropcoords", frame=self.frame_dict["config_func"], button_dict=self.button_dict,
                          row=2, column=2, text="set cropcoords", 
                          command=lambda: setCropcoords(self, self.entry_dict["cropcoords"], self.canvas_dict["window"]))
        createEntry(key="threshold_loop", frame=self.frame_dict["config_func"], entry_dict=self.entry_dict, 
                       row=3, column=0, text="threshold:", entry_width=entry_width, pad=pad, textvariable=tk.DoubleVar(value=0.0))
        createEntry(key="threshold_loop_framenum", frame=self.frame_dict["config_func"], entry_dict=self.entry_dict, 
                       row=3, column=2, text="threshold frame number:", entry_width=entry_width, pad=pad, textvariable=tk.IntVar(value=60))
        createEntry(key="threshold_dlclh", frame=self.frame_dict["config_func"], entry_dict=self.entry_dict, 
                       row=4, column=0, text="threshold DLC likelihood:", entry_width=entry_width, pad=pad, textvariable=tk.DoubleVar(value=0.6))

        # "Camera Config"項目の作成
        createLabel(key="config_camera", frame=self.frame_dict["config_camera"], label_dict=self.label_dict, 
                         row=0, column=0, text="Camera Config:")

        createEntry(key="fps", frame=self.frame_dict["config_camera"], entry_dict=self.entry_dict, 
                       row=1, column=0, text="fps:", entry_width=entry_width, pad=pad, textvariable=tk.DoubleVar(value=60.0))
        createEntry(key="width", frame=self.frame_dict["config_camera"], entry_dict=self.entry_dict, 
                       row=2, column=0, text="width:", entry_width=entry_width, pad=pad, textvariable=tk.IntVar(value=1280))
        createEntry(key="height", frame=self.frame_dict["config_camera"], entry_dict=self.entry_dict, 
                       row=3, column=0, text="height:", entry_width=entry_width, pad=pad, textvariable=tk.IntVar(value=1024))
        createEntry(key="offsetx", frame=self.frame_dict["config_camera"], entry_dict=self.entry_dict, 
                       row=4, column=0, text="offsetx:", entry_width=entry_width, pad=pad, textvariable=tk.IntVar(value=0))
        createEntry(key="offsety", frame=self.frame_dict["config_camera"], entry_dict=self.entry_dict, 
                       row=5, column=0, text="offsety:", entry_width=entry_width, pad=pad, textvariable=tk.IntVar(value=0))
        createEntry(key="gain", frame=self.frame_dict["config_camera"], entry_dict=self.entry_dict, 
                       row=6, column=0, text="gain:", entry_width=entry_width, pad=pad, textvariable=tk.DoubleVar(value=12.0))
        createEntry(key="exposure_time", frame=self.frame_dict["config_camera"], entry_dict=self.entry_dict, 
                       row=7, column=0, text="exposure_time:", entry_width=entry_width, pad=pad, textvariable=tk.DoubleVar(value=2000))
        
        # "HoG Config"項目の作成
        createLabel(key="config_hog", frame=self.frame_dict["config_camera"], label_dict=self.label_dict, 
                         row=0, column=2, text="HoG Vector Config:")

        createEntry(key="hog_orientations", frame=self.frame_dict["config_camera"], entry_dict=self.entry_dict, 
                       row=1, column=2, text="orientations:", entry_width=entry_width, pad=pad, textvariable=tk.IntVar(value=8))
        createEntry(key="hog_pixels_per_cell", frame=self.frame_dict["config_camera"], entry_dict=self.entry_dict, 
                       row=2, column=2, text="pixels_per_cell:", entry_width=entry_width, pad=pad, textvariable=tk.IntVar(value=16))
        createEntry(key="hog_cells_per_block", frame=self.frame_dict["config_camera"], entry_dict=self.entry_dict, 
                       row=3, column=2, text="cells_per_block:", entry_width=entry_width, pad=pad, textvariable=tk.IntVar(value=1))
        createEntry(key="hog_smoothing_window", frame=self.frame_dict["config_camera"], entry_dict=self.entry_dict, 
                       row=4, column=2, text="smoothing window:", entry_width=entry_width, pad=pad, textvariable=tk.IntVar(value=16))
        
        # --- frame_dict["capture"] ---
        # "Caputre One shot"ボタンの作成
        createButton(key="capture_single", frame=self.frame_dict["capture"], button_dict=self.button_dict,
                          row=0, column=0, text="Capture Single shot", 
                          command=lambda: singleCaptureAndShow(self, self.canvas_dict["window"]))
        # "play"ボタンの作成
        createButton(key="play", frame=self.frame_dict["capture"], button_dict=self.button_dict,
                          row=0, column=1, text="Play", 
                          command=self.startCaptureMovie)
        # FPSラベル
        createLabel(key="fps", frame=self.frame_dict["capture"], label_dict=self.label_dict, 
                         row=1, column=1, text="0 fps")
        # "capture movie"ボタンの作成
        createButton(key="capture_movie", frame=self.frame_dict["capture"], button_dict=self.button_dict,
                          row=0, column=2, text="Capture Movie", 
                          command=lambda: self.startCaptureMovie(save=True, crop=False))
        # captureラベル
        createLabel(key="capture_movie", frame=self.frame_dict["capture"], label_dict=self.label_dict, 
                         row=1, column=2, text="")
        # "Capture with DLC"ボタンの作成
        createButton(key="capture_dlc_start", frame=self.frame_dict["capture"], button_dict=self.button_dict,
                          row=3, column=0, text="Start Capture with DLC", 
                          command=self.startCaptureMovieDLC)
        # "Capture by Bpod Cue"ボタンの作成
        createButton(key="capture_bpod_start", frame=self.frame_dict["capture"], button_dict=self.button_dict,
                          row=3, column=1, text="Start Capture with Bpod", 
                          command=self.startCaptureMovieBpod)
        # "Move Video"ボタンの作成
        createButton(key="move_video", frame=self.frame_dict["capture"], button_dict=self.button_dict,
                          row=4, column=0, text="Move Video Files", 
                          command=self.moveVideoFiles)
        # "Exit"ボタンの作成
        createButton(key="exit", frame=self.frame_dict["capture"], button_dict=self.button_dict,
                          row=4, column=1, text="Exit", 
                          command=self.exitGUI)
        
        # --- frame_dict["image"] ---
        # "Window"項目の作成
        initializeCanvasCropImage(self, self.frame_dict["image"], row=0, column=0, width=1280, height=1024, key_canvas="window", key_window="main")

        # カメラ
        self.capture_flag = False
        if self.camera_name == "Basler":
            self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        
        # DLC-Live
        self.dlc_flag = False
        
        # Closed loop
        self.func_flag = False
        
        # シリアル通信
        self.serial = False
        
        # デバッグ用
        if self.debug:
            self.window_dict["debug"] = VariableInspector(self)
            self.window_dict["main"].protocol("WM_DELETE_WINDOW", self.exitGUI)
        
    """
    Function Closed-Loop
    """

    # thresholdと判定
    def judgeThresholdValue(self, value, threshold):
        bool_loop = value > threshold
        return bool_loop
    # window分の平均を計算し、一定時間閾値以上となったらTrue
    def judgeThresholdWindowContinuousValue(self, value, threshold, window, bool_loop_framenum, threshold_framenum):
        # windowの更新
        window = np.append(np.delete(window, 0), value)
        mean_window = np.mean(window)
        # 平均値が閾値以上か
        if mean_window > threshold:
            bool_loop_framenum += 1
            # 一定時間続いたか
            if bool_loop_framenum > threshold_framenum:
                bool_loop = True
            else:
                bool_loop = False
        else:
            bool_loop_framenum = 0
            bool_loop = False
        return bool_loop, window, bool_loop_framenum
        
    # DLC-Liveのセット
    def setDLCLive(self):
        self.dlc_model = filedialog.askdirectory() # DLCモデルの選択
        self.dlc_live = DLCLive(self.dlc_model, display=False) # Processorはセットしない
        frame = singleCapture(self) # init用の画像
        self.dlc_live.init_inference(frame) # 初期化
        self.threshold_dlclh = float(self.entry_dict["threshold_dlclh"].get()) # likelihood threshold 
    # DLC-Liveで予測した座標を取得
    def getDLCLivePose(self, image):
        self.dlc_live.get_pose(image)
        pose = self.dlc_live.pose
        return pose
    # 指定したポイントの座標を抜き出し、likelihoodが閾値以上であるかの判定も返す
    def chooseDLCCoords(self, pose, list_poseidx, threshold):
        pose = pose[list_poseidx]
        likelihood = pose[:, 2]
        bool_dlclh = np.all(likelihood > threshold) # 全ての座標のlikelihoodが閾値以上であるか
        return pose, bool_dlclh
    
    # 座標4点を通る円の半径
    def findCircleCenterandRadius(self, points):
        xa, ya = points[0]
        xb, yb = points[1]
        xc, yc = points[2]
        xd, yd = points[3]

        D = 2 * (xa * (yb - yc) + xb * (yc - ya) + xc * (ya - yb))
        Ux = ((xa**2 + ya**2) * (yb - yc) + (xb**2 + yb**2) * (yc - ya) + (xc**2 + yc**2) * (ya - yb)) / D
        Uy = ((xa**2 + ya**2) * (xc - xb) + (xb**2 + yb**2) * (xa - xc) + (xc**2 + yc**2) * (xb - xa)) / D

        r = np.sqrt((xa - Ux)**2 + (ya - Uy)**2)

        return (Ux, Uy), r

    
    # 値と、その値がthreshold以上であるかの判定を返す
    
    # Fake用, 0.1秒待つ
    def wait100msec(self, img):
        value_loop = 0
        bool_loop = False
        time.sleep(0.1)
        return value_loop, bool_loop
    
    # 画素平均値が閾値以上の明るさとなったら
    def valuePixel(self, img):
        value_loop = np.mean(img)
        bool_loop = self.judgeThresholdValue(value, self.threshold_loop)
        return value_loop, bool_loop
        
    # HoGVector L2ノルムの変化量
    # 直近window個の平均値が閾値以上となり、それがthreshold_sec秒以上続いたら
    def valueHoGVectorL2normDelta(self, img):
        orientations = int(self.entry_dict["hog_orientations"].get())
        pixels_per_cell = int(self.entry_dict["hog_pixels_per_cell"].get())
        cells_per_block = int(self.entry_dict["hog_cells_per_block"].get())
        
        hog_features = hogFeatures(img, orientations, pixels_per_cell, cells_per_block)
        
        # 1フレーム目はスキップ
        if not len(self.hog_tmp) > 0:
            value_loop = 0
            bool_loop = False
        else:
            hog_delta = hog_features - self.hog_tmp # 差分
            hog_delta_abs = np.zeros(hog_features.shape[:2]) # 差分の絶対値
            for h in range(len(hog_delta)):
                for w in range(len(hog_delta[h])):
                    hog_delta_abs[h][w] = np.linalg.norm(hog_delta[h][w][0][0], ord=2) # L2 norm

            hog_delta_abs_mean = np.mean(hog_delta_abs)
            value_loop = hog_delta_abs_mean
            
            bool_loop, self.hog_window, self.hog_bool_loop_framenum = self.judgeThresholdWindowContinuousValue(value_loop, 
                                                                                                               self.threshold_loop, 
                                                                                                               self.hog_window, 
                                                                                                               self.hog_bool_loop_framenum, 
                                                                                                               self.threshold_loop_framenum)
        
        # HoGの更新
        self.hog_tmp = hog_features
        return value_loop, bool_loop
        
    # DLCliveでpupilを追跡、閾値以上となったら
    def dlcLivePupil(self, img):
        pose = self.getDLCLivePose(img)
        coords, bool_dlclh = self.chooseDLCCoords(pose=pose, list_poseidx=[2,3,4,5], threshold=self.threshold_dlclh)
        coords = ((coords[0][0], coords[0][1]), (coords[1][0], coords[1][1]), (coords[2][0], coords[2][1]), (coords[3][0], coords[3][1]))
        if bool_dlclh: # likelihoodが十分高いか
            center, radius = self.findCircleCenterandRadius(coords)
            value_loop = radius
            bool_loop = self.judgeThresholdValue(value_loop, self.threshold_loop)
        else:
            value_loop = 0
            bool_loop = False
        return value_loop, bool_loop, pose
        
    """
    Function
    """  
    # USBカメラかBaslerカメラか選ぶ
    def chooseCameraName(self, list_camera=["USB", "Basler"], toplevel=False):
        def okCameraName(self):
            # リストボックスから選択された項目を取得
            self.camera_name = listbox.get(listbox.curselection())
            if len(self.camera_name) > 0:
                window.destroy()  # ウィンドウを閉じる
            else:
                messagebox.showerror("Error", "Please select camera.")
        
        if toplevel:
            window = tk.Toplevel()
        else:
            window = tk.Tk()
        window.title("Choose Camera")
        # ウィンドウを最前面に表示
        window.lift()
        window.attributes('-topmost', True)
        # リストボックス
        listbox = tk.Listbox(window, height=3)
        for i, camera_name in enumerate(list_camera):
            listbox.insert(i+1, camera_name)
        listbox.pack()
        # OKボタン
        button = tk.Button(window, text="OK", command=lambda: okCameraName(self))
        button.pack()
        window.mainloop()
        
    # 検証用の動画撮影
    def captureMovie(self, save, crop):
        if save:
            path_save = filedialog.asksaveasfilename(defaultextension=".npy", 
                                         filetypes=[("NPY files", "*.npy"),
                                                    ("AVI files", "*.avi"),
                                                    ("MP4 files", "*.mp4"),
                                                    ("All files", "*.")]) # 保存先
            if len(path_save) == 0:
                self.capture_flag = False
            if path_save[-3:] == "avi" or path_save[-3:] == "mp4":
                self.save_as_video = True
                h, w = int(self.entry_dict["height"].get()), int(self.entry_dict["width"].get())
                fourcc = cv2.VideoWriter_fourcc(*"MP4V") # mp4
                fps = float(self.entry_dict["fps"].get())
                out = cv2.VideoWriter(path_save, fourcc, fps, (w, h), isColor=False) # 白黒で出力
            else:
                self.save_as_video = False
            updateLabel(self.label_dict["capture_movie"], text="capturing...", color="black") # ラベル更新
            
        # 検証用
        self.ims = []
        cv2.namedWindow('Esc or Right Click : stop capture')
        cv2.setMouseCallback('Esc or Right Click : stop capture', self.stopCaptureMovieWithMouse)
            
        while self.capture_flag:
            
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                img = image.GetArray()
                # crop範囲の描画
                if crop:
                    if len(self.cropcoords) > 0:
                        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
                        cv2.rectangle(img, (self.cropcoords[0], self.cropcoords[2]), (self.cropcoords[1], self.cropcoords[3]), (0, 0, 255), 2)
                cv2.imshow('Esc or Right Click : stop capture', img)

                t = time.time()
                self.t_array.append(t)
                
                # 動画保存
                if self.save_as_video:
                    out.write(img)
                if save:
                    self.ims += [img]
                    
                # escキーでbreak
                k = cv2.waitKey(1)
                if k == 27:
                    self.capture_flag = False
                    break
                
            grabResult.Release()
        if save:
            if self.save_as_video:
                out.release()
            else:
                # npyとして保存
                self.ims = np.array(self.ims)
                np.save(path_save, self.ims)
            updateLabel(self.label_dict["capture_movie"], text="captured !", color="red") # ラベル更新
        del self.ims
        self.camera.StopGrabbing()
        self.camera.Close()
        cv2.destroyAllWindows()
        showFPS(t_array=self.t_array, label=self.label_dict["fps"])
        
    # 右クリックでキャプチャ終了
    def stopCaptureMovieWithMouse(self, event, x, y, flags, param):
        # 右クリックでウィンドウを閉じる
        if event == cv2.EVENT_RBUTTONDOWN:
            self.capture_flag = False

        
    # DLC検証用の動画撮影
    def captureMovieDLC(self):
        while self.capture_flag:
            
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                img = image.GetArray()
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) # カラーに変換
                # crop範囲の描画
                if len(self.cropcoords) > 0:
                    cv2.rectangle(img, (self.cropcoords[0], self.cropcoords[2]), (self.cropcoords[1], self.cropcoords[3]), (0, 0, 255), 2)
                # DLC liveで座標取得
                pose = self.getDLCLivePose(img)
                # 取得した座標上に丸を描く
                for i, pos in enumerate(pose):
                    if pos[2] > self.threshold_dlclh:
                        rgb = self.cmap_dlc(i)[:-1]  # RGB色を取得
                        bgr = tuple(int(val * 255) for val in rgb[::-1])  # BGRに変換し、整数に変換
                        cv2.circle(img, (pos[0], pos[1]), 10, bgr, -1)  # 円の描画
                
                cv2.imshow('Esc : stop capture', img)

                t = time.time()
                self.t_array.append(t)
                    
                # escキーでbreak
                k = cv2.waitKey(1)
                if k == 27:
                    self.capture_flag = False
                    break
                
            grabResult.Release()
        self.camera.StopGrabbing()
        self.camera.Close()
        cv2.destroyAllWindows()
        showFPS(t_array=self.t_array, label=self.label_dict["fps"])
        
    # Captureの開始
    def startCaptureMovie(self, save=False, crop=True):
        # カメラセット
        self.camera.Open()
        getCameraConfigDict(self)
        setCamera(self)
        
        # キャプチャ時間
        self.t_array = []
        
        if not self.capture_flag:
            self.capture_flag = True
            self.save_as_video = False
            self.camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)
            self.captureMovie(save, crop)
            
    # Captureの開始 DLC検証
    def startCaptureMovieDLC(self):
        # カメラセット
        self.camera.Open()
        getCameraConfigDict(self)
        setCamera(self)
        # DLC Liveセット
        self.setDLCLive()
        
        self.all_joints_names = self.dlc_live.cfg["all_joints_names"]
        self.cmap_dlc = cm.get_cmap("jet", len(self.all_joints_names))
        
        # キャプチャ時間
        self.t_array = []
        
        if not self.capture_flag:
            self.capture_flag = True
            self.camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)
            self.captureMovieDLC()
            
    # Captureの開始 with Bpod
    def startCaptureMovieBpod(self):
        # ボタン操作不可
        self.button_dict["capture_bpod_start"].config(state=tk.DISABLED)
        # prefixが空であるとき注意喚起
        prefix_check = checkEntryEmpty(self.pulldown_dict["savedir_prefix"], "prefix check", "prefix is empty, Continue?")
        if not prefix_check:
            # ボタン操作可
            self.button_dict["capture_bpod_start"].config(state=tk.NORMAL)
            return
        
        # Closed-loopで実行する関数
        self.func_name = self.pulldown_dict["function_loop"].get()
        if self.func_name:
            # Closed loop関数が指定されているとき注意喚起(事故防止)
            func_check = messagebox.askokcancel("Closed-loop check", f"Closed-loop function: {self.func_name}, Continue?")
            if not func_check:
                # ボタン操作可
                self.button_dict["capture_bpod_start"].config(state=tk.NORMAL)
                return
            
            self.func_flag = True
            self.dlc_flag = False
            self.func_loop = self.dict_func[self.func_name]
            self.threshold_loop = float(self.entry_dict["threshold_loop"].get())
            self.threshold_loop_framenum = int(self.entry_dict["threshold_loop_framenum"].get())
            
            # DLC-Liveを使用するか
            if self.func_name[:3] == "dlc":
                self.dlc_flag = True
                self.setDLCLive()
                
        # カメラセット
        self.camera.Open()
        getCameraConfigDict(self)
        setCamera(self)
        self.camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)
        
        # 全トライアルのキャプチャ時間
        self.t_array_trial = []
        # トライアルごとのclosed loopの値、判定
        self.value_loop_array_trial = []
        self.bool_loop_array_trial = []
        self.dlc_pose_array_trial = []
        
        # トライアル数
        self.trial = 0
        
        # SSH接続でラズパイにpyファイルの実行命令を送信
        # pyfilename = "MonitorGPIOInput"
        # executePythonWithSSH(pyfilename)
        
        # Bpodからのシグナルを待機
        self.waitBpodSignal()
                        
    # トライアル毎の撮影
    def captureMovieTrial(self):
        # Closed_loopで前の画像との差分を利用する場合 HoGVector用
        self.hog_tmp = np.array(())
        self.hog_window = np.zeros(int(self.entry_dict["hog_smoothing_window"].get()))
        self.hog_bool_loop_framenum = 0
        
        # 動画出力の準備
        exportdir_movie = self.exportdir + f"/trial{self.trial}.mp4"
        h, w = int(self.entry_dict["height"].get()), int(self.entry_dict["width"].get())
        fourcc = cv2.VideoWriter_fourcc(*"MP4V") # mp4
        fps = float(self.entry_dict["fps"].get())
        out = cv2.VideoWriter(exportdir_movie, fourcc, fps, (w, h), 0) # 白黒で出力
        
        firstframe = True
        
        while self.capture_flag_trial:
            # Bpodからのシグナル
            signal = self.ser_receiver.readline().decode('shift-jis').strip()
            # キャプチャ終了
            if signal == "0":
                self.capture_flag_trial = False
                self.ser_sender.write(b"trialend\n") # trial終了の信号
                print(f"Trial{self.trial} : Capture End")
                break
            # 撮影中断
            if signal == "9":
                self.closeCapture(exportconfig=False)
                print("Capture Interrupted.")
                break
                
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

            # 撮影
            if grabResult.GrabSucceeded():
                # 1フレーム目(例外処理)
                if firstframe:
                    self.ser_sender.write(b"trialstart\n") # 最初のフレームだけ信号を送る
                    firstframe = False
                
                # 画像取得
                image = self.converter.Convert(grabResult)
                img = image.GetArray()

                # 動画書き込み
                out.write(img)

                # 1フレーム当たりのキャプチャ時間
                t = time.time()
                self.t_array_trial.append(t)
                
                # Closed-loopで実行する関数
                if self.func_flag:
                    img = cropImage(img, self.cropcoords) # クロップ
                    # DLCを使用するか
                    if self.dlc_flag:
                        value_loop, bool_loop, dlc_pose = self.func_loop(img)
                        self.dlc_pose_array_trial.append(dlc_pose)
                    else:
                        value_loop, bool_loop = self.func_loop(img)
                    self.value_loop_array_trial.append(value_loop)
                    self.bool_loop_array_trial.append(bool_loop)
                    # arduino sender cameraに送信
                    if bool_loop:
                        self.ser_sender.write(b"on\n")
                    else:
                        self.ser_sender.write(b"off\n")
                        
            grabResult.Release()
        # キャプチャ終了
        out.release() # 動画出力
        # csv出力
        self.exportTrialCSV()
        self.exportDLCPoseCSV()

        # arduino sender cameraに送信
        self.ser_sender.write(b"exportend\n")
        print(f"Trial{self.trial} : Exported !")
    
    # シリアル通信の開始
    def openSerial(self, timeout=None):
        com_receive = self.pulldown_dict["comport_receiver"].get()
        com_sender = self.pulldown_dict["comport_sender"].get()
        baudrate = self.entry_dict["baudrate"].get()
        self.ser_receiver = serial.Serial(com_receive, baudrate, timeout=timeout)
        self.ser_sender = serial.Serial(com_sender, baudrate, timeout=timeout)
        self.serial = True
    # シリアル通信の切断
    def closeSerial(self):
        self.ser_receiver.close()
        self.ser_sender.close()
        self.serial = False
    
    # Bpodからの信号を待つ。
    def waitBpodSignal(self):
        # シリアル通信開始
        self.openSerial()
        # カメラ撮影フラグ
        self.capture_flag_trial = False
        # 実験サブジェクト名
        print("Waiting subject name ...")
        self.subjectname = self.ser_receiver.readline().decode('shift_jis').strip()
        print("Subject name :", self.subjectname)
        # 実験名を受け取るまではserialのtimeoutをNone(信号を受け取るまで待機)
        # 受け取った後は、timeout=0で待機しない
        self.ser_receiver.timeout = 0
        self.ser_sender.timeout = 0
        # 保存先
        self.exportdir = self.entry_dict["savedir"].get() + "/VideoFile_Rawdata_" + self.subjectname
        if len(self.pulldown_dict["savedir_prefix"].get()) > 0:
            self.exportdir = self.exportdir + f"_{self.pulldown_dict['savedir_prefix'].get()}"
        if not os.path.exists(self.exportdir):
            os.makedirs(self.exportdir)
        print("Directory name :", self.exportdir)

        
        # 毎フレームBpodからのシグナルをチェックする。"0"が来たら撮影を終了する
        # "0" : Trialの撮影終了
        # "1" : Trialの撮影開始
        # "9" : 実験終了、プログラムを終了する。
        while self.serial:
            # Bpodからのシグナル
            signal = self.ser_receiver.readline().decode('shift-jis').strip()
            
            # キャプチャの判定
            if self.capture_flag_trial:
                self.captureMovieTrial()
            else:
                # キャプチャ開始
                if signal == "1":
                    self.capture_flag_trial = True
                    self.trial += 1
                    print(f"Trial{self.trial} : Capture Start")
                # 実験終了
                elif signal == "9":
                    self.closeCapture()
                    print("Capture Finished !")
                    break
        # カメラ終了
        self.camera.StopGrabbing()
        self.camera.Close()
        
    # 撮影終了、中断
    def closeCapture(self, exportconfig=True):
        # arduino sender cameraに送信
        self.ser_sender.write(b"interrupt\n")
        self.capture_flag_trial = False
        # シリアル通信終了
        self.closeSerial()
        # コンフィグ出力
        if exportconfig:
            self.exportConfig()
        # ボタン操作可
        self.button_dict["capture_bpod_start"].config(state=tk.NORMAL)
        
    # trialごとにcsv出力 
    # タイムスタンプ,、closed loopの値、判定
    def exportTrialCSV(self):
        exportdir = self.exportdir + f"/trial{self.trial}.csv"
        # Closed-loopで実行するか
        if self.func_flag:
            col = ["timestamp", "valueClosedLoop", "boolClosedLoop"]
            value = np.array([self.t_array_trial, self.value_loop_array_trial, self.bool_loop_array_trial,]).T
        else:
            col = ["timestamp"]
            value = np.array([self.t_array_trial,]).T
        df = pd.DataFrame(value, columns=col)
        df.to_csv(exportdir, index=None)
                
        # 初期化
        self.t_array_trial = []
        self.value_loop_array_trial = []
        self.bool_loop_array_trial = []
        
    # DLC pose csvの出力
    def exportDLCPoseCSV(self):
        if self.dlc_flag:
            bodyparts = [i for i in self.dlc_live.cfg["all_joints_names"] for _ in range(3)]
            coords = ["x", "y", "likelihood"] * len(self.dlc_live.cfg["all_joints_names"])
            exportdir = self.exportdir + f"/dlcPose_trial{self.trial}.csv"
            df = pd.DataFrame()
            df["bodyparts"] = bodyparts
            df["coords"] = coords
            pose = np.array(self.dlc_pose_array_trial)
            pose = pose.reshape(pose.shape[0], -1)
            pose = pd.DataFrame(pose).T
            df = pd.concat([df, pose], axis=1)
            df = df.T
            df.to_csv(exportdir, header=None)
        # 初期化
        self.dlc_pose_array_trial = []
        
    # Configファイルの出力
    def exportConfig(self):
        exportdir = self.exportdir + f"/config.csv"
        config = {}
        for key, value in self.entry_dict.items():
            try:
                config[key] = value.get()
            except AttributeError:
                config[key] = value
        if self.func_flag: # Closed loop あり
            config["ClosedLoopFunc"] = self.func_name
        if self.dlc_flag: # DLC live あり
            config["DLCmodel"] = self.dlc_model
        config = pd.DataFrame(config, index=[0]).T
        config.to_csv(exportdir, header=None)
        
    # 動画ファイルの移動
    def moveVideoFiles(self):
        # 保存先
        savedir = self.entry_dict["savedir"].get()
        # 移動先
        movedst = self.entry_dict["movedst"].get()
        
        # config.csvが含まれない(正常に出力されていない)、名前に"_"を含まないフォルダ、FakeSubjectを省く
        list_folder = []
        for folder in getMatchedPaths(getAllSubDirectories(savedir, depth=1), list_str_include=["_"], list_str_exclude=["FakeSubject"]):
            if getMatchedPaths(getAllSubFiles(folder), list_str_include=["/config.csv"]):
                list_folder += [folder]
        
        print(list_folder)
        
        print("Files moving ...")

        for folder_path in tqdm(list_folder, desc="Moving files", ncols=100):
            try:
                # フォルダの移動
                folder_name = folder_path.rsplit("/")[-1]
                # 新記名ルール
                if "__ID" in folder_name:
                    # フォルダ名からセッション名、個体名(2桁アルファベット+2桁数字)、日付(6桁数字)を取得
                    name_session = f"database_{folder_name.split('_')[5]}"
                    name_mouse = folder_name.split('_')[2]
                    _ID = folder_name.split("_ID_")[-1] # _ID_以降の文字列
                    name_date = _ID.split('_')[1][-6:]
                # 旧記名ルール
                else:
                    # フォルダ名からセッション名、個体名(2桁アルファベット+2桁数字)、日付(6桁数字)を取得
                    name_session = f"database_{folder_name.split('_')[3]}"
                    name_mouse = folder_name.split('_')[2]
                    name_date = folder_name.split('_')[4][-6:]
                
                # ファイル名チェック
                if not re.match(r'^\d{6}$', name_date):
                    raise ValueError("name_dateが6桁の数字ではありません。")
                if not re.match(r'^[A-Za-z]{2}\d{2,3}$', name_mouse):
                    raise ValueError("name_mouseが2桁のアルファベット+2,3桁の数字ではありません。")

                # 新しいフォルダパスを作成
                path_new = os.path.join(movedst, name_session)
                path_new = os.path.join(path_new, name_mouse)
                path_new = os.path.join(path_new, name_date).replace("\\", "/")

                print(folder_path)
                print("->", os.path.join(path_new, folder_name).replace("\\", "/"))
                print()

                # 新しいフォルダを作成（すでに存在する場合はスキップ）
                os.makedirs(path_new, exist_ok=True)

                # フォルダを移動
                shutil.move(folder_path, path_new)
            # 条件に合わないフォルダはパス
            except Exception as e:
                print(folder_path)
                print(e)
                print("passed")
                print()
                pass

        # 条件を満たさないフォルダはvideo_tmpフォルダに移動
        print("Files moving to tmp folder...")
        list_folder_tmp = getMatchedPaths(getAllSubDirectories(savedir, depth=1), list_str_include=["_"], list_str_exclude=["video_tmp"])
        print(list_folder_tmp)
        for src_folder in tqdm(list_folder_tmp, desc="Moving tmp files", ncols=100):
            dst_folder = src_folder.rsplit("/",1)[0] + "/video_tmp/" + src_folder.rsplit("/",1)[1]
            print(src_folder)
            print("->", dst_folder)
            shutil.move(src_folder, dst_folder)

        print("Files move finished !")
        
    # 終了
    def exitGUI(self):
        try:
            if self.camera.IsGrabbing():
                self.camera.Close()
        except AttributeError:
            pass
        if self.debug:
            self.window_dict["debug"].destroy()
        self.window_dict["main"].destroy()
        
if __name__ == '__main__':
    camera_gui = CameraGUI(debug=False, toplevel=False)
    camera_gui.window_dict["main"].mainloop()

c:\Users\share\anaconda3\envs\camera\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [3]:
camera_gui.camera

<pypylon.pylon.InstantCamera; proxy of <Swig Object of type 'Pylon::CInstantCamera *' at 0x000002C7B0F9E8D0> >

In [ ]:
import pylon

pylon.InstantCamera

Type:           InstantCamera
String form:    <pypylon.pylon.InstantCamera; proxy of <Swig Object of type 'Pylon::CInstantCamera *' at 0x000002C7B0F9E8D0> >
File:           c:\users\share\anaconda3\envs\camera\lib\site-packages\pypylon\pylon.py
Docstring:     
Provides convenient access to a camera device.  

*   Establishes a single access point for accessing camera functionality.  
*   The class can be used off the shelf without any parameters. The camera uses
    a default configuration for the camera device. This can be overridden.  
*   Handles Pylon device lifetime. This can be overridden.  
*   Handles opening and closing of a Pylon device automatically.  
*   Handles chunk data parsing automatically returning the chunk data in the
    grab result.  
*   Handles event grabbing automatically providing a convenient interface for
    event callbacks. This can be overridden.  
*   Handles physical camera device removal.  
*   Handles the creation, reuse, and destruction of buffers. 